In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import sys
import pandas as pd

sys.path.append('../../../param_search')
import param_search as ps

ps.set_verbose(False)
ps.set_backend('slurm')

In [3]:
csv_path = '2025-11-02_selected.csv'
sel = pd.read_csv(csv_path)
sel

,subject,metadata.category.set,metadata.category.len,metadata.category.set_f,metadata.category.len_f,metadata.category.valid,metadata.category.reasons,metadata.unit.value,metadata.unit.valid,metadata.unit.reasons,...,artifacts.valid,artifacts.reasons,valid,reasons,artifacts.scene.exc,paths.source_mask.exc,source_path,job_name,job_id,dims_error
0,wss.100f39dce7690f59efb94709f30ce0d2,"{'Recliner', 'Chair'}",2,"{'Recliner', 'Chair'}",2,True,(),0.012947,True,(),...,True,(),True,(),NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,0103f8f8d5c25bf7,35732758,7.294375e-08
1,wss.101354f9d8dede686f7b08d9de913afe,"{'Speaker', '_Attributes'}",2,{'Speaker'},1,True,(),0.013620,True,(),...,True,(),True,(),NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,0103f8f8d5c25bf7,35732758,1.319075e-08
2,wss.1029fd5d71a06ea32ea98d69e91ba870,{'ChestOfDrawers'},1,{'ChestOfDrawers'},1,True,(),0.019631,True,(),...,True,(),True,(),NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,0103f8f8d5c25bf7,35732758,8.319422e-08
3,wss.102a6b7809f4e51813842bc8ef6fe18,"{'Desktop', 'Computer'}",2,"{'Desktop', 'Computer'}",2,True,(),0.025252,True,(),...,True,(),True,(),NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,0103f8f8d5c25bf7,35732758,1.077898e-07
4,wss.107637b6bdf8129d4904d89e9169817b,{'Couch'},1,{'Couch'},1,True,(),0.022625,True,(),...,True,(),True,(),NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,0103f8f8d5c25bf7,35732758,7.520307e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,wss.d81f943065391492a7e760c0b399301,"{'FoodItem', 'Bowl', '_StanfordSceneDBModels',...",4,"{'FoodItem', 'Bowl', 'FruitBowl'}",3,True,(),0.001381,True,(),...,True,(),False,"('missing dims',)",NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,5c023e2f832d4dcf,35732849,NaN
1112,wss.d8d4004791c4f61b80fa98b5eeb7036c,"{'_StanfordSceneDBModels', 'Book'}",2,{'Book'},1,True,(),0.000002,True,(),...,True,(),False,"('missing dims',)",NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,5c023e2f832d4dcf,35732849,NaN
1113,wss.e8add7acb28e2505fd40810f48721e8b,"{'Bed', 'DoubleBed'}",2,"{'Bed', 'DoubleBed'}",2,True,(),0.009245,True,(),...,True,(),False,"('missing dims',)",NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,5c023e2f832d4dcf,35732849,NaN
1114,wss.f6420a3a3addd027f67be63aca8832dd,"{'Recliner', 'Chair'}",2,"{'Recliner', 'Chair'}",2,True,(),0.027262,True,(),...,True,(),False,"('missing dims',)",NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,8786de37533664e5,35732850,NaN


In [97]:
base_dir = '2025-11-12_train'

template = '''\
#!/bin/bash -l
#SBATCH --job-name={job_name}
#SBATCH --account=asc170022p
#SBATCH --partition=GPU-shared
#SBATCH -w v034
#SBATCH --mem=22750M
#SBATCH --gres=gpu:1
#SBATCH -t 24:00:00
set -eo pipefail

LP=$PROJECT/lung-project
NB=$PROJECT/lung-project/notebooks/shapenet

mamba activate /ocean/projects/asc170022p/mtragoza/mambaforge/envs/warp

python $LP/train.py \\
    --dataset shapenet \\
    --data_root $LP/data/ShapeNetSem \\
    --variant {variant} \\
    --config {config} \\
    --subject $NB/{subject} \\
    --output {output_path} \\
    --supervised {supervised} \\
    --learning_rate {learning_rate} \\
    --tv_reg_weight {tv_reg_weight}

'''
name_format = '{params_hash}'

grid = ps.param_grid(
    subject='2025-11-05_optimized.csv',
    variant='2025-11-02',
    config='2025-11-05_config.json',
    supervised=['true'],
    learning_rate=[3e-3, 1e-4, 3e-5],
    tv_reg_weight=[0.0],
) + ps.param_grid(
    subject='2025-11-05_optimized.csv',
    variant='2025-11-02',
    config='2025-11-05_config.json',
    supervised=['false'],
    learning_rate=[3e-3, 1e-4, 3e-5],
    tv_reg_weight=[3e-3, 1e-4, 3e-5],
)
len(grid)

12

In [98]:
%autoreload
try:
    jobs = ps.setup(base_dir, template, name_format, grid, overwrite=True)
except OSError:
    jobs = ps.load(base_dir)

jobs

,job_name,job_state,n_submits,job_id,node_id,runtime,stdout,stderr,base_dir,work_dir,...,stdout_path,stderr_path,params_json,params_hash,params.subject,params.variant,params.config,params.supervised,params.learning_rate,params.tv_reg_weight
0,31574372a30ea704,NEW,0,<NA>,<NA>,<NA>,<NA>,<NA>,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,<NA>,<NA>,"{""config"": ""2025-11-05_config.json"", ""learning...",31574372a30ea704,2025-11-05_optimized.csv,2025-11-02,/ocean/projects/asc170022p/mtragoza/lung-proje...,true,0.00300,0.00000
1,b108c732fd87866c,NEW,0,<NA>,<NA>,<NA>,<NA>,<NA>,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,<NA>,<NA>,"{""config"": ""2025-11-05_config.json"", ""learning...",b108c732fd87866c,2025-11-05_optimized.csv,2025-11-02,/ocean/projects/asc170022p/mtragoza/lung-proje...,true,0.00010,0.00000
2,f7137f58f4dc9b11,NEW,0,<NA>,<NA>,<NA>,<NA>,<NA>,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,<NA>,<NA>,"{""config"": ""2025-11-05_config.json"", ""learning...",f7137f58f4dc9b11,2025-11-05_optimized.csv,2025-11-02,/ocean/projects/asc170022p/mtragoza/lung-proje...,true,0.00003,0.00000
3,877d698ccef39738,NEW,0,<NA>,<NA>,<NA>,<NA>,<NA>,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,<NA>,<NA>,"{""config"": ""2025-11-05_config.json"", ""learning...",877d698ccef39738,2025-11-05_optimized.csv,2025-11-02,/ocean/projects/asc170022p/mtragoza/lung-proje...,false,0.00300,0.00300
4,4b620b928cfe5751,NEW,0,<NA>,<NA>,<NA>,<NA>,<NA>,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,<NA>,<NA>,"{""config"": ""2025-11-05_config.json"", ""learning...",4b620b928cfe5751,2025-11-05_optimized.csv,2025-11-02,/ocean/projects/asc170022p/mtragoza/lung-proje...,false,0.00300,0.00010
5,d9a94d4eff9ec108,NEW,0,<NA>,<NA>,<NA>,<NA>,<NA>,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,<NA>,<NA>,"{""config"": ""2025-11-05_config.json"", ""learning...",d9a94d4eff9ec108,2025-11-05_optimized.csv,2025-11-02,/ocean/projects/asc170022p/mtragoza/lung-proje...,false,0.00300,0.00003
6,815bf148a9464fb7,NEW,0,<NA>,<NA>,<NA>,<NA>,<NA>,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,<NA>,<NA>,"{""config"": ""2025-11-05_config.json"", ""learning...",815bf148a9464fb7,2025-11-05_optimized.csv,2025-11-02,/ocean/projects/asc170022p/mtragoza/lung-proje...,false,0.00010,0.00300
7,05b7857409a5f464,NEW,0,<NA>,<NA>,<NA>,<NA>,<NA>,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,<NA>,<NA>,"{""config"": ""2025-11-05_config.json"", ""learning...",05b7857409a5f464,2025-11-05_optimized.csv,2025-11-02,/ocean/projects/asc170022p/mtragoza/lung-proje...,false,0.00010,0.00010
8,aeed649c0c1c664b,NEW,0,<NA>,<NA>,<NA>,<NA>,<NA>,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,<NA>,<NA>,"{""config"": ""2025-11-05_config.json"", ""learning...",aeed649c0c1c664b,2025-11-05_optimized.csv,2025-11-02,/ocean/projects/asc170022p/mtragoza/lung-proje...,false,0.00010,0.00003
9,bd2c681923d5fdff,NEW,0,<NA>,<NA>,<NA>,<NA>,<NA>,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,<NA>,<NA>,"{""config"": ""2025-11-05_config.json"", ""learning...",bd2c681923d5fdff,2025-11-05_optimized.csv,2025-11-02,/ocean/projects/asc170022p/mtragoza/lung-proje...,false,0.00003,0.00300


In [99]:
%autoreload
jobs = ps.recover(jobs)
jobs = ps.status(jobs)
jobs = ps.history(jobs)
jobs.groupby(['job_state']).count()

,job_name,n_submits,job_id,node_id,runtime,stdout,stderr,base_dir,work_dir,script_path,...,params.variant,params.config,params.supervised,params.learning_rate,params.tv_reg_weight,array_idx,last_live_at,state_source,finalized,finalized_at
job_state,,,,,,,,,,,,,,,,,,,,,
FAILED,9,9,9,9,9,0,0,9,9,9,...,9,9,9,9,9,0,0,9,9,9
RECOVERED,3,3,3,0,0,0,0,3,3,3,...,3,3,3,3,3,0,0,0,0,0


In [100]:
jobs = ps.collect(jobs)
jobs

,job_name,job_state,n_submits,job_id,node_id,runtime,stdout,stderr,base_dir,work_dir,...,params.learning_rate,params.tv_reg_weight,array_idx,last_live_at,state_source,finalized,finalized_at,output_exists,output_fsize,output_mtime
0,31574372a30ea704,RECOVERED,1,35917753,<NA>,<NA>,[Epoch 0 | Train batch 421/739] loss = 9.9988e...,"File ""/ocean/projects/asc170022p/mtragoza/lu...",/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,0.00300,0.00000,NaN,NaN,NaN,NaN,NaN,False,<NA>,<NA>
1,b108c732fd87866c,RECOVERED,1,35917754,<NA>,<NA>,[Epoch 0 | Train batch 420/739] loss = 9.9986e...,"Traceback (most recent call last):\n File ""/o...",/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,0.00010,0.00000,NaN,NaN,NaN,NaN,NaN,False,<NA>,<NA>
2,f7137f58f4dc9b11,RECOVERED,1,35917755,<NA>,<NA>,[Epoch 0 | Train batch 419/739] loss = 9.9868e...,"File ""/ocean/projects/asc170022p/mtragoza/lu...",/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,0.00003,0.00000,NaN,NaN,NaN,NaN,NaN,False,<NA>,<NA>
3,877d698ccef39738,FAILED,1,35917756,v005,00:05:05,[Epoch 0 | Val batch 51/108] loss = 1.9994e+09...,^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,0.00300,0.00300,NaN,NaN,history,True,2025-11-12T06:20:10,False,<NA>,<NA>
4,4b620b928cfe5751,FAILED,1,35917757,v024,00:08:27,[Epoch 0 | Train batch 40/739] loss = 9.9045e-...,return _launch_integrate_kernel(\n ...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,0.00300,0.00010,NaN,NaN,history,True,2025-11-12T06:20:10,False,<NA>,<NA>
5,d9a94d4eff9ec108,FAILED,1,35917758,v024,00:08:27,[Epoch 0 | Train batch 45/739] loss = 1.0261e+...,return _launch_integrate_kernel(\n ...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,0.00300,0.00003,NaN,NaN,history,True,2025-11-12T06:20:10,False,<NA>,<NA>
6,815bf148a9464fb7,FAILED,1,35917759,v024,00:08:27,[Epoch 0 | Train batch 36/739] loss = 1.3133e+...,return _launch_integrate_kernel(\n ...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,0.00010,0.00300,NaN,NaN,history,True,2025-11-12T06:20:10,False,<NA>,<NA>
7,05b7857409a5f464,FAILED,1,35917760,v024,00:08:33,[Epoch 0 | Train batch 47/739] loss = 8.5024e-...,return _launch_integrate_kernel(\n ...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,0.00010,0.00010,NaN,NaN,history,True,2025-11-12T06:20:10,False,<NA>,<NA>
8,aeed649c0c1c664b,FAILED,1,35917761,v019,00:08:54,[Epoch 0 | Train batch 56/739] loss = 4.9243e+...,return _launch_integrate_kernel(\n ...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,0.00010,0.00003,NaN,NaN,history,True,2025-11-12T06:20:10,False,<NA>,<NA>
9,bd2c681923d5fdff,FAILED,1,35917762,v018,00:08:52,[Epoch 0 | Train batch 63/739] loss = 9.9632e-...,return _launch_integrate_kernel(\n ...,/ocean/projects/asc170022p/mtragoza/lung-proje...,/ocean/projects/asc170022p/mtragoza/lung-proje...,...,0.00003,0.00300,NaN,NaN,history,True,2025-11-12T06:20:10,False,<NA>,<NA>


In [91]:
j = jobs[jobs.job_state == 'RECOVERED'].iloc[2]
print(j.stderr); j

  File "/ocean/projects/asc170022p/mtragoza/lung-project/train.py", line 139, in <module>
    main(sys.argv[1:])
  File "/ocean/projects/asc170022p/mtragoza/lung-project/train.py", line 135, in main
    trainer.train(args.epochs)
  File "/ocean/projects/asc170022p/mtragoza/lung-project/project/training.py", line 67, in train
    self.run_train_phase(run_eval=eval_on_train)
  File "/ocean/projects/asc170022p/mtragoza/lung-project/project/training.py", line 76, in run_train_phase
    for i, batch in enumerate(self.train_loader):
  File "/ocean/projects/asc170022p/mtragoza/mambaforge/envs/warp/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 626, in __next__
    with torch.autograd.profiler.record_function(self._profile_name):
  File "/ocean/projects/asc170022p/mtragoza/mambaforge/envs/warp/lib/python3.11/site-packages/torch/autograd/profiler.py", line 705, in __exit__
    torch.ops.profiler._record_function_exit._RecordFunction(record)
  File "/ocean/projects/asc170022p

job_name                                                 f7137f58f4dc9b11
job_state                                                       RECOVERED
n_submits                                                               1
job_id                                                           35917755
node_id                                                              <NA>
runtime                                                              <NA>
stdout                  [Epoch 0 | Train batch 419/739] loss = 9.9868e...
stderr                    File "/ocean/projects/asc170022p/mtragoza/lu...
base_dir                /ocean/projects/asc170022p/mtragoza/lung-proje...
work_dir                /ocean/projects/asc170022p/mtragoza/lung-proje...
script_path             /ocean/projects/asc170022p/mtragoza/lung-proje...
output_path             /ocean/projects/asc170022p/mtragoza/lung-proje...
log_dir                 /ocean/projects/asc170022p/mtragoza/lung-proje...
stdout_path             /ocean/project

In [101]:
jobs.loc[:, 'job_id'] = pd.NA
jobs = ps.submit(jobs)